In [2]:
from gbs_functions import montrealer, cumulant_calculator, symmetric_M, hermitian_N
from sympy import expand, simplify

In [3]:
import numpy as np
np.set_printoptions(linewidth=200)

In [4]:
from gbs_functions_A import moment as moment_A # Moments in terms of the A matrix
from gbs_functions_A import cumulant as cumulant_A # Cumulants in terms of the A matrix

In [5]:
moment_A([1,1,1],displacement=False)

A[1, 2]*A[3, 4]*A[5, 6] + A[1, 2]*A[3, 5]*A[4, 6] + A[1, 2]*A[3, 6]*A[4, 5] + A[1, 3]*A[2, 4]*A[5, 6] + A[1, 3]*A[2, 5]*A[4, 6] + A[1, 3]*A[2, 6]*A[4, 5] + A[1, 4]*A[2, 3]*A[5, 6] + A[1, 4]*A[2, 5]*A[3, 6] + A[1, 4]*A[2, 6]*A[3, 5] + A[1, 5]*A[2, 3]*A[4, 6] + A[1, 5]*A[2, 4]*A[3, 6] + A[1, 5]*A[2, 6]*A[3, 4] + A[1, 6]*A[2, 3]*A[4, 5] + A[1, 6]*A[2, 4]*A[3, 5] + A[1, 6]*A[2, 5]*A[3, 4]

In [6]:
c4 = cumulant_A([1,1,1,1],displacement=False)

In [7]:
c3 = cumulant_A([1,1,1],displacement=False)

In [8]:
c5 = cumulant_A([1,1,1,1,1],displacement=False)

In [9]:
c2 = cumulant_A([1,1],displacement=False)

In [10]:
c1 = cumulant_A([1],displacement=False)

In [11]:
m1 = moment_A([1],displacement=False)

In [12]:
m2 = moment_A([1,1],displacement=False) 

In [13]:
m3 = moment_A([1,1,1],displacement=False)

In [14]:
m4 = moment_A([1,1,1,1],displacement=False)

In [15]:
m5 = moment_A([1,1,1,1,1],displacement=False)

In [16]:
len(c1.as_terms()[0])

1

In [17]:
ms = [m1,m2,m3,m4,m5]
cs = [c1,c2,c3,c4,c5]

In [19]:
from scipy.special import factorial, factorial2

In [20]:
for i in range(5):
    lm = len(ms[i].as_terms()[0])
    lc = len(cs[i].as_terms()[0])
    print(str(i+1) + " " + str(lm) + " " + str(lc) +" "+ str(factorial(i)*2**i))

1 1 1 1.0
2 3 2 2.0
3 15 8 8.0
4 105 48 48.0
5 945 384 384.0


# The things below are more speculative

In [64]:
from thewalrus.reference import pmp
from itertools import product, permutations

In [65]:
def pair_selection(n):
    for binary in map(''.join, product('01', repeat=int(n/2)-1)):
        yield '0'+binary

In [66]:
def flip(l):
    return [l[1],l[0]]

In [67]:
def cyclic_permutation(n):
    iterable = permutations(range(int(n/2)-1))
    for iter in iterable:
        yield iter + (int(n/2)-1,iter[0])

In [68]:
def dicting(x):
    return {x[i]:x[i+1] for i in range(len(x)-1) }

In [69]:
def cycle_to_permutation(cycle):
    n = len(cycle)-1
    return np.array([dicting(cycle)[i] for i in range(n)])

In [75]:
n =6
p = [[2*i+1,2*i+2] for i in range(n//2)]


In [76]:
p

[[1, 2], [3, 4], [5, 6]]

In [77]:
import numpy as np

In [78]:
for x in pair_selection(n):
    pp = np.array([flip(p[i]) if digit==1 else p[i] for i,digit in enumerate(x)]).T
    indices = np.copy(pp)
    for cycle in cyclic_permutation(n):
        perm = cycle_to_permutation(cycle)
        print(np.array([indices[0],indices[1][perm]]).T)

[[1 4]
 [3 6]
 [5 2]]
[[1 6]
 [3 2]
 [5 4]]
[[1 4]
 [3 6]
 [5 2]]
[[1 6]
 [3 2]
 [5 4]]
[[1 4]
 [3 6]
 [5 2]]
[[1 6]
 [3 2]
 [5 4]]
[[1 4]
 [3 6]
 [5 2]]
[[1 6]
 [3 2]
 [5 4]]


In [79]:
c3

A[1, 2]*A[3, 4]*A[5, 6] + A[1, 2]*A[3, 5]*A[4, 6] + A[1, 3]*A[2, 4]*A[5, 6] + A[1, 3]*A[2, 6]*A[4, 5] + A[1, 5]*A[2, 3]*A[4, 6] + A[1, 5]*A[2, 6]*A[3, 4] + A[1, 6]*A[2, 3]*A[4, 5] + A[1, 6]*A[2, 4]*A[3, 5]

In [80]:
from thewalrus.reference import pmp
from itertools import product, permutations
from gbs_functions import symmetric_M
from sympy import Matrix, expand
from operator import itemgetter
from gbs_functions import montrealer
import copy
from time import time
import numpy as np

In [81]:
def pair_selection(n):
    for binary in map(''.join, product('01', repeat=int(n/2)-1)):
        yield '0'+binary

def flip(l):
    return [l[1],l[0]]

def cyclic_permutation(n):
    iterable = permutations(range(int(n/2)-1))
    for iter in iterable:
        yield iter + (int(n/2)-1,iter[0])

def dicting(x):
    return {x[i]:x[i+1] for i in range(len(x)-1) }

def cycle_to_permutation(cycle):
    n = len(cycle)-1
    return np.array([dicting(cycle)[i] for i in range(n)])

def term_builder(M, iterable, conjugate=False):
    term = 1
    for i in iterable:
        term *= M[i[0]][i[1]]
    return term.conjugate() if conjugate else term

In [82]:
def matching_terms(p):
    n = 2*len(p)
    for x in pair_selection(n):
        pp = np.array([flip(p[i]) if digit=='1' else p[i] for i,digit in enumerate(x)]).T
        indices = np.copy(pp)
        for cycle in cyclic_permutation(n):
            perm = cycle_to_permutation(cycle)
            yield np.array([indices[0],indices[1][perm]]).T

def montrealer_combinatorics(M):
    n = len(M)
    equation = 0
    for p in pmp(tuple(range(n))):
        #p = list(map(list,p))
        term = term_builder(M, p, conjugate=True)
        for x in matching_terms(p):
            equation += term*term_builder(M, x, conjugate=False)
    return equation

In [112]:
def tobenamed(M):
    n = len(M)//2
    equation = 0
    #p = [(2*i, 2*i+1) for i in range(n)]
    p = [(2*i, (2*i-1) % (2*n)) for i in range(n)]
    term = 1
    for x in matching_terms(p):
        equation += term*term_builder(M, x, conjugate=False)
    return equation

In [113]:
from gbs_functions_A import symmetric_A

In [118]:
tobenamed(symmetric_A(3))

A[1, 2]*A[3, 4]*A[5, 6] + A[1, 2]*A[3, 5]*A[4, 6] + A[1, 3]*A[2, 4]*A[5, 6] + A[1, 3]*A[2, 5]*A[4, 6] + A[1, 4]*A[2, 5]*A[3, 6] + A[1, 4]*A[2, 6]*A[3, 5] + A[1, 5]*A[2, 4]*A[3, 6] + A[1, 5]*A[2, 6]*A[3, 4]

In [100]:
[x for x in cyclic_permutation(8)]

[(0, 1, 2, 3, 0),
 (0, 2, 1, 3, 0),
 (1, 0, 2, 3, 1),
 (1, 2, 0, 3, 1),
 (2, 0, 1, 3, 2),
 (2, 1, 0, 3, 2)]

In [103]:
n = 6
[i for i in range(n,0,-1)]

[6, 5, 4, 3, 2, 1]

In [110]:
[(2*i, (2*i-1) % (2*n)) for i in range(n)]

[(0, 11), (2, 1), (4, 3), (6, 5), (8, 7), (10, 9)]

In [123]:
[x for x in pmp(tuple(range(4)))]

[((0, 1), (2, 3)), ((0, 2), (1, 3)), ((0, 3), (1, 2))]

# Even more speculative

In [173]:
def cumulant_local(M):
    r"""Blarg
    """
    n = len(M)
    iter_set = pmp(tuple(range(n)))

    tot_sum = 0
    nhalf = n//2
    
    for i in iter_set:
        add_up = True
        for pair in i:
            if pair[1] - pair[0] == nhalf:
                add_up = False
                break
        if add_up is True:
            result = 1

            for j in i:
                result = result * M[j]

            tot_sum = tot_sum + result

    return tot_sum

In [179]:
m4 - cumulant_local(symmetric_A(4))

A[1, 2]*A[3, 5]*A[4, 8]*A[6, 7] + A[1, 2]*A[3, 6]*A[4, 8]*A[5, 7] + A[1, 2]*A[3, 7]*A[4, 5]*A[6, 8] + A[1, 2]*A[3, 7]*A[4, 6]*A[5, 8] + A[1, 2]*A[3, 7]*A[4, 8]*A[5, 6] + A[1, 3]*A[2, 5]*A[4, 8]*A[6, 7] + A[1, 3]*A[2, 6]*A[4, 5]*A[7, 8] + A[1, 3]*A[2, 6]*A[4, 7]*A[5, 8] + A[1, 3]*A[2, 6]*A[4, 8]*A[5, 7] + A[1, 3]*A[2, 7]*A[4, 8]*A[5, 6] + A[1, 4]*A[2, 5]*A[3, 7]*A[6, 8] + A[1, 4]*A[2, 6]*A[3, 5]*A[7, 8] + A[1, 4]*A[2, 6]*A[3, 7]*A[5, 8] + A[1, 4]*A[2, 6]*A[3, 8]*A[5, 7] + A[1, 4]*A[2, 8]*A[3, 7]*A[5, 6] + A[1, 5]*A[2, 3]*A[4, 6]*A[7, 8] + A[1, 5]*A[2, 3]*A[4, 7]*A[6, 8] + A[1, 5]*A[2, 3]*A[4, 8]*A[6, 7] + A[1, 5]*A[2, 4]*A[3, 6]*A[7, 8] + A[1, 5]*A[2, 4]*A[3, 7]*A[6, 8] + A[1, 5]*A[2, 4]*A[3, 8]*A[6, 7] + A[1, 5]*A[2, 6]*A[3, 4]*A[7, 8] + A[1, 5]*A[2, 6]*A[3, 7]*A[4, 8] + A[1, 5]*A[2, 6]*A[3, 8]*A[4, 7] + A[1, 5]*A[2, 7]*A[3, 4]*A[6, 8] + A[1, 5]*A[2, 7]*A[3, 6]*A[4, 8] + A[1, 5]*A[2, 7]*A[3, 8]*A[4, 6] + A[1, 5]*A[2, 8]*A[3, 4]*A[6, 7] + A[1, 5]*A[2, 8]*A[3, 6]*A[4, 7] + A[1, 5]*A[2, 8

In [181]:
cumulant_local(symmetric_A(4)) - c4

A[1, 2]*A[3, 4]*A[5, 6]*A[7, 8] + A[1, 2]*A[3, 8]*A[4, 7]*A[5, 6] + A[1, 3]*A[2, 4]*A[5, 7]*A[6, 8] + A[1, 3]*A[2, 8]*A[4, 6]*A[5, 7] + A[1, 4]*A[2, 3]*A[5, 8]*A[6, 7] + A[1, 4]*A[2, 7]*A[3, 6]*A[5, 8] + A[1, 6]*A[2, 5]*A[3, 4]*A[7, 8] + A[1, 6]*A[2, 5]*A[3, 8]*A[4, 7] + A[1, 7]*A[2, 4]*A[3, 5]*A[6, 8] + A[1, 7]*A[2, 8]*A[3, 5]*A[4, 6] + A[1, 8]*A[2, 3]*A[4, 5]*A[6, 7] + A[1, 8]*A[2, 7]*A[3, 6]*A[4, 5]

In [175]:
c4

A[1, 2]*A[3, 4]*A[5, 7]*A[6, 8] + A[1, 2]*A[3, 4]*A[5, 8]*A[6, 7] + A[1, 2]*A[3, 5]*A[4, 6]*A[7, 8] + A[1, 2]*A[3, 5]*A[4, 7]*A[6, 8] + A[1, 2]*A[3, 6]*A[4, 5]*A[7, 8] + A[1, 2]*A[3, 6]*A[4, 7]*A[5, 8] + A[1, 2]*A[3, 8]*A[4, 5]*A[6, 7] + A[1, 2]*A[3, 8]*A[4, 6]*A[5, 7] + A[1, 3]*A[2, 4]*A[5, 6]*A[7, 8] + A[1, 3]*A[2, 4]*A[5, 8]*A[6, 7] + A[1, 3]*A[2, 5]*A[4, 6]*A[7, 8] + A[1, 3]*A[2, 5]*A[4, 7]*A[6, 8] + A[1, 3]*A[2, 7]*A[4, 5]*A[6, 8] + A[1, 3]*A[2, 7]*A[4, 6]*A[5, 8] + A[1, 3]*A[2, 8]*A[4, 5]*A[6, 7] + A[1, 3]*A[2, 8]*A[4, 7]*A[5, 6] + A[1, 4]*A[2, 3]*A[5, 6]*A[7, 8] + A[1, 4]*A[2, 3]*A[5, 7]*A[6, 8] + A[1, 4]*A[2, 5]*A[3, 6]*A[7, 8] + A[1, 4]*A[2, 5]*A[3, 8]*A[6, 7] + A[1, 4]*A[2, 7]*A[3, 5]*A[6, 8] + A[1, 4]*A[2, 7]*A[3, 8]*A[5, 6] + A[1, 4]*A[2, 8]*A[3, 5]*A[6, 7] + A[1, 4]*A[2, 8]*A[3, 6]*A[5, 7] + A[1, 6]*A[2, 3]*A[4, 5]*A[7, 8] + A[1, 6]*A[2, 3]*A[4, 7]*A[5, 8] + A[1, 6]*A[2, 4]*A[3, 5]*A[7, 8] + A[1, 6]*A[2, 4]*A[3, 8]*A[5, 7] + A[1, 6]*A[2, 7]*A[3, 4]*A[5, 8] + A[1, 6]*A[2, 7

In [137]:
m3 - c3

A[1, 2]*A[3, 6]*A[4, 5] + A[1, 3]*A[2, 5]*A[4, 6] + A[1, 4]*A[2, 3]*A[5, 6] + A[1, 4]*A[2, 5]*A[3, 6] + A[1, 4]*A[2, 6]*A[3, 5] + A[1, 5]*A[2, 4]*A[3, 6] + A[1, 6]*A[2, 5]*A[3, 4]